# Mini-sdAIgen
A lightweight, focused downloader for ComfyUI workflows on Kaggle/Colab.

### Instructions
1. Run the **Setup** cell to clone the repository (sparse checkout), install dependencies, **and install ComfyUI**.
2. Run the **Widgets** cell to configure your API tokens and Model URLs.
3. Run the **Download** cell to start downloading.
4. Run the **Launch** cell to start ComfyUI.

In [ ]:
# 1. Setup Environment
import os
import sys
import shutil

# Optimized Clone: Fetch ONLY the mini-sdAIgen folder
if not os.path.exists('mini-sdAIgen'):
    print("Cloning mini-sdAIgen repository (Sparse Checkout)...")
    
    # 1. Prepare temp dir
    temp_dir = "temp_clone"
    os.makedirs(temp_dir, exist_ok=True)
    original_dir = os.getcwd()
    os.chdir(temp_dir)
    
    try:
        # 2. Git Sparse Checkout sequence
        !git init
        !git remote add origin https://github.com/Faturrachman-dev/notebook.git
        !git config core.sparseCheckout true
        !echo "mini-sdAIgen/" >> .git/info/sparse-checkout
        !git pull --depth 1 origin main
        
        # 3. Move target folder out
        os.chdir(original_dir)
        if os.path.exists(f"{temp_dir}/mini-sdAIgen"):
            # Move the folder to root
            if os.path.exists("mini-sdAIgen"):
                 shutil.rmtree("mini-sdAIgen")
            shutil.move(f"{temp_dir}/mini-sdAIgen", ".")
            print("Clone successful.")
        else:
            print("Error: mini-sdAIgen folder not found in remote repo. Please check repo structure.") 
            
    except Exception as e:
        os.chdir(original_dir)
        print(f"Clone failed: {e}")
    finally:
        # 4. Cleanup
        if os.path.exists(temp_dir):
            if sys.platform == 'win32':
                !rmdir /S /Q temp_clone
            else:
                !rm -rf temp_clone

# Ensure repo is in path
repo_path = os.path.abspath("mini-sdAIgen")
if repo_path not in sys.path:
    sys.path.append(repo_path)

# Run setup script
!python mini-sdAIgen/setup.py

# Fix: Upgrade SQLAlchemy for ComfyUI compatibility
!pip install --upgrade sqlalchemy
# Install pyngrok for tunneling
!pip install pyngrok

# Install ComfyUI if missing
if not os.path.exists('/root/ComfyUI'):
    print("Installing ComfyUI...")
    !git clone https://github.com/comfyanonymous/ComfyUI /root/ComfyUI
    
    # \"Lightweight\" Install: Skip Torch/Nvidia libs if pre-installed (Kaggle usually has them)
    print("Installing ComfyUI Dependencies (Smart)...")
    req_path = '/root/ComfyUI/requirements.txt'
    if os.path.exists(req_path):
        with open(req_path, 'r') as f:
            reqs = f.readlines()
            
        # Filter out heavy/existing packages
        filtered_reqs = []
        for r in reqs:
            pkg = r.strip().split('=')[0].split('<')[0].split('>')[0] # simplistic parse
            if pkg.lower() in ['torch', 'torchvision', 'torchaudio', 'cupy-cuda12x', 'cupy-cuda11x']:
                print(f"Skipping {pkg} (assuming pre-installed in KaggleEnv)")
            else:
                filtered_reqs.append(r)
        
        # Write temp requirements
        with open('temp_reqs.txt', 'w') as f:
            f.writelines(filtered_reqs)
            
        !pip install -r temp_reqs.txt
        !rm temp_reqs.txt
    else:
        print("requirements.txt not found, skipping dependency install.")

    # Install ComfyUI-Manager (Essential)
    manager_path = '/root/ComfyUI/custom_nodes/ComfyUI-Manager'
    if not os.path.exists(manager_path):
        print("Installing ComfyUI-Manager...")
        !git clone https://github.com/ltdrdata/ComfyUI-Manager {manager_path}
    else:
        print("ComfyUI-Manager already installed.")

else:
    print("ComfyUI already installed.")

In [ ]:
# 2. Configure Widgets
import launch
launch.show_widgets()

In [ ]:
# 3. Start Download
launch.run_download()

In [ ]:
# 4. Launch ComfyUI with Ngrok
import json
import os
from pyngrok import ngrok, conf

if os.path.exists('/root/ComfyUI/main.py'):
    # 1. Setup Ngrok if token exists
    settings_path = 'settings.json'
    if os.path.exists(settings_path):
        with open(settings_path, 'r') as f:
            settings = json.load(f)
            ngrok_token = settings.get('ngrok_token')
            
        if ngrok_token:
            print("Starting Ngrok Tunnel...")
            conf.get_default().auth_token = ngrok_token
            try:
                public_url = ngrok.connect(8188).public_url
                print(f"\n>>> ComfyUI Public URL: {public_url} <<<\n")
            except Exception as e:
                print(f"Ngrok Error: {e}")
        else:
            print("No Ngrok token found in settings.json. UI will not be accessible remotely.")
    
    # 2. Launch ComfyUI
    !python /root/ComfyUI/main.py --listen --port 8188
else:
    print("ComfyUI not found at /root/ComfyUI/main.py. Check your paths.")